In [60]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import DecimalType
from datetime import date
import os

In [67]:
spark = SparkSession.builder.master('local').appName('app').getOrCreate()
#spark.conf.set("spark.sql.shuffle.partitions", 5)  # avoid unneeded shuffling

cloud_storage_path = "wasbs://test@guidedcapstonesa.blob.core.windows.net"
# trade_common_df = spark.read.parquet(f"{cloud_storage_path}/trade")

INPUT_DIRECTORY = "output_dir"
file_input = 'file://' + os.getcwd() + '/' + INPUT_DIRECTORY 
trade_common_df = spark.read.parquet(file_input)
trades_df = trade_common_df.where(trade_common_df.partition == 'T')                           
quotes_extended_df = trade_common_df.where(trade_common_df.partition == 'Q')\
                           .withColumn('mov_avg_trade_pr', lit(None).cast(DecimalType(10, 2)))

trades_df.createOrReplaceTempView("trades")
quotes_df.createOrReplaceTempView("quotes_extended")

trades_df.printSchema()
quotes_df.printSchema()


root
 |-- trade_dt: date (nullable = true)
 |-- rec_type: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- event_tm: timestamp (nullable = true)
 |-- event_seq_nb: integer (nullable = true)
 |-- arrival_tm: timestamp (nullable = true)
 |-- trade_pr: decimal(10,2) (nullable = true)
 |-- bid_pr: decimal(10,2) (nullable = true)
 |-- bid_size: integer (nullable = true)
 |-- ask_pr: decimal(10,2) (nullable = true)
 |-- ask_size: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- partition: string (nullable = true)

root
 |-- trade_dt: date (nullable = true)
 |-- rec_type: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- event_tm: timestamp (nullable = true)
 |-- event_seq_nb: integer (nullable = true)
 |-- arrival_tm: timestamp (nullable = true)
 |-- trade_pr: decimal(10,2) (nullable = true)
 |-- bid_pr: decimal(10,2) (nullable = true)
 |-- bid_size: integer 

In [72]:
sql2 = """
SELECT trade_dt, symbol, exchange, event_tm, trade_pr,
LAG(trade_pr, 1) OVER (partition by symbol, exchange order by event_tm) as trade_pr_prior
from trades;
"""

sql3 = """
SELECT *, 
mean(trade_pr) OVER  
(partition by symbol, exchange order by event_tm
RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW)
as mov_avg_trade_pr FROM trades;
"""

sql4 = """

SELECT trade_dt, symbol, exchange, trade_pr_close,
lag(trade_pr_close,1) OVER (partition by symbol, exchange order by trade_dt) as trade_pr_close_prior
from 
(
    SELECT trade_dt, symbol, exchange, event_tm, trade_pr,
    nth_value(trade_pr,1) OVER (partition by symbol, exchange, trade_dt order by event_tm desc) as trade_pr_close,
    row_number() OVER (partition by symbol, exchange, trade_dt order by event_tm desc) as row
    from trades
) a

WHERE row = 1
ORDER BY trade_dt, symbol, exchange;
"""



spark.sql(sql3).createOrReplaceTempView("trades_extended")

sql_union = """
select * from trades_extended
union
select * from quotes_extended
order by event_tm
"""

spark.sql(sql_union)\
.select('trade_dt', 'rec_type', 'symbol', 'exchange', 'event_tm', 'trade_pr', 'mov_avg_trade_pr')\
.createOrReplaceTempView("quote_trade_extended")

In [74]:
sql_prior = """
SELECT trade_dt, symbol, exchange, event_tm, trade_pr,
last(trade_pr, True) OVER (partition by symbol, exchange order by event_tm) as prior_trade_pr
from quote_trade_extended;
"""

spark.sql(sql_prior).show(100,False)

+----------+------+--------+-----------------------+--------+--------------+
|trade_dt  |symbol|exchange|event_tm               |trade_pr|prior_trade_pr|
+----------+------+--------+-----------------------+--------+--------------+
|2020-08-05|SYMC  |NYSE    |2020-08-05 09:35:05.228|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 09:41:59.433|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 09:51:34.255|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:00:47.821|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:09:18.246|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:18:13.448|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:25:51.729|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:32:28.771|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:38:59.521|null    |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:44:56.492|160.88  |160.88        |

In [50]:
spark.sql(sql2).show(20,False)

+----------+------+--------+-----------------------+--------+--------------+
|trade_dt  |symbol|exchange|event_tm               |trade_pr|trade_pr_prior|
+----------+------+--------+-----------------------+--------+--------------+
|2020-08-05|SYMC  |NYSE    |2020-08-05 10:44:56.492|160.88  |null          |
|2020-08-05|SYMC  |NYSE    |2020-08-05 11:58:06.861|157.03  |160.88        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 13:10:55.919|157.25  |157.03        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 14:32:26.063|160.94  |157.25        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 15:48:06.73 |159.17  |160.94        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 17:01:38.379|161.15  |159.17        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 18:20:43.709|158.18  |161.15        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 19:37:55.659|161.13  |158.18        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 20:48:19.795|159.72  |161.13        |
|2020-08-05|SYMC  |NYSE    |2020-08-05 21:52:38.607|160.62  |159.72        |

In [22]:
trade_common_df.where(trade_common_df.partition == 'T').show(120, truncate=False)

+----------+--------+------+--------+-----------------------+------------+-------------------+--------+------+--------+------+--------+----+---------+
|trade_dt  |rec_type|symbol|exchange|event_tm               |event_seq_nb|arrival_tm         |trade_pr|bid_pr|bid_size|ask_pr|ask_size|line|partition|
+----------+--------+------+--------+-----------------------+------------+-------------------+--------+------+--------+------+--------+----+---------+
|2020-08-06|T       |SYMA  |NASDAQ  |2020-08-06 10:42:21.079|10          |2020-08-06 09:30:00|78.93   |null  |null    |null  |null    |null|T        |
|2020-08-06|T       |SYMA  |NASDAQ  |2020-08-06 12:00:29.595|20          |2020-08-06 09:30:00|77.10   |null  |null    |null  |null    |null|T        |
|2020-08-06|T       |SYMA  |NASDAQ  |2020-08-06 13:09:29.883|30          |2020-08-06 09:30:00|78.31   |null  |null    |null  |null    |null|T        |
|2020-08-06|T       |SYMA  |NASDAQ  |2020-08-06 14:27:08.62 |40          |2020-08-06 09:30:00|

In [48]:
trade_common_df.printSchema()

root
 |-- trade_dt: date (nullable = true)
 |-- rec_type: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- event_tm: timestamp (nullable = true)
 |-- event_seq_nb: integer (nullable = true)
 |-- arrival_tm: timestamp (nullable = true)
 |-- trade_pr: decimal(10,2) (nullable = true)
 |-- bid_pr: decimal(10,2) (nullable = true)
 |-- bid_size: integer (nullable = true)
 |-- ask_pr: decimal(10,2) (nullable = true)
 |-- ask_size: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- partition: string (nullable = true)



In [42]:
trade_common_df.select('trade_dt').distinct().collect()

[Row(trade_dt=datetime.date(2021, 6, 22)),
 Row(trade_dt=datetime.date(2021, 6, 23))]

In [43]:
trade_common_df.count()

240

In [28]:
# Generalize for any number of days

all_data = spark.createDataFrame([], COMMON_SCHEMA)  # empty dataframe to accumulate all files

dates = [date(21,6,22), date(21,6,23)]

trades = []
for dt in dates:
    in_file = cloud_storage_path + '/trade/trade_dt=' + dt.isoformat() 
    trades.append(spark.read.parquet(in_file))
    
    
    

wasbs://test@guidedcapstonesa.blob.core.windows.net/trade/trade_dt=0021-06-22
wasbs://test@guidedcapstonesa.blob.core.windows.net/trade/trade_dt=0021-06-23


In [38]:
trade_common_df.createOrReplaceGlobalTempView('quote')


In [39]:
spark.sql('describe  table extended global_temp.quote ').show(50)

+------------+-------------+-------+
|    col_name|    data_type|comment|
+------------+-------------+-------+
|    trade_dt|         date|   null|
|      symbol|       string|   null|
|    exchange|       string|   null|
|    event_tm|    timestamp|   null|
|event_seq_nb|          int|   null|
|  arrival_tm|    timestamp|   null|
|      bid_pr|decimal(10,2)|   null|
|    bid_size|          int|   null|
|      ask_pr|decimal(10,2)|   null|
|    ask_size|          int|   null|
|    quote_dt|         date|   null|
+------------+-------------+-------+



In [18]:
spark.sql("select count(*) from quote;")

DataFrame[count(1): bigint]

In [19]:
trade_common_df.take(5)

[Row(trade_dt=datetime.date(2020, 8, 5), symbol='SYMB', exchange='NASDAQ', event_tm=datetime.datetime(2020, 8, 5, 13, 3, 51, 427000), event_seq_nb=30, arrival_tm=datetime.datetime(2020, 8, 5, 9, 30), bid_pr=None, bid_size=None, ask_pr=None, ask_size=None),
 Row(trade_dt=datetime.date(2020, 8, 5), symbol='SYMC', exchange='NASDAQ', event_tm=datetime.datetime(2020, 8, 5, 18, 15, 15, 631000), event_seq_nb=70, arrival_tm=datetime.datetime(2020, 8, 5, 9, 30), bid_pr=None, bid_size=None, ask_pr=None, ask_size=None),
 Row(trade_dt=datetime.date(2020, 8, 6), symbol='SYMC', exchange='NASDAQ', event_tm=datetime.datetime(2020, 8, 6, 18, 59, 56, 853000), event_seq_nb=80, arrival_tm=datetime.datetime(2020, 8, 6, 9, 30), bid_pr=None, bid_size=None, ask_pr=None, ask_size=None),
 Row(trade_dt=datetime.date(2020, 8, 5), symbol='SYMA', exchange='NASDAQ', event_tm=datetime.datetime(2020, 8, 5, 14, 22, 41, 390000), event_seq_nb=40, arrival_tm=datetime.datetime(2020, 8, 5, 9, 30), bid_pr=None, bid_size=None